This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


print(API_KEY)

PNJWaxLghJ6uoBkSntPK


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2020-11-02&end_date=2020-11-02&key=' + API_KEY 
response = requests.get(url)
response.content.strip()

b'{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.\\u003cbr\\u003e\\u003cbr\\u003eTrading System: Xetra\\u003cbr\\u003e\\u003cbr\\u003eISIN: DE0005313704","refreshed_at":"2020-12-01T14:48:09.907Z","newest_available_date":"2020-12-01","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":null,"transform":null,"column_index":null,"start_date":"2020-11-02","end_date":"2020-11-02","data":[["2020-11-02",111.3,111.6,108.1,108.1,null,362.0,39844.5,null,null,null]],"collapse":null,"order":null,"database_id":6129}}'

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [31]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY 
response = requests.get(url)
stock_info = json.loads(response.content.decode('utf-8'))

In [56]:
highest = 0
lowest = stock_info['dataset']['data'][0][1]

largestOneDay = 0
largestTwoDays = 0

previousClose = stock_info['dataset']['data'][0][4]

avgVol = 0
vol_collection = []

In [57]:
#print('entries: ', len(stock_info['dataset']['data']))
#print(stock_info['dataset']['column_names'])

for daily in stock_info['dataset']['data']:
    #print(daily)
    
    #find the highest and lowest
    if(daily[1] is not None and daily[1] > highest):
        highest = daily[1]
    if(daily[1] is not None and daily[1] < lowest):
        lowest = daily[1]   
    
    change = daily[2] - daily[3]
    if(change > largestOneDay):
        largestOneDay = change
    
    currClose = daily[4]
    closingDiff = abs(currClose - previousClose)
    if(closingDiff > largestTwoDays):
        largestTwoDays = closingDiff
    previousClose = currClose
    
    #trade volume
    vol_collection.append(daily[6])
    avgVol += daily[6]
    
avgVol = avgVol/365
vol_collection.sort()

In [58]:
print('higest opening price is', highest)
print('lowest opening price is', lowest)
print('largest change in one day is', largestOneDay)
print('largest change between 2 days is', largestTwoDays)
print('average trading volume, based on 365 days and not opening days:',avgVol)
print('median trading volume is',vol_collection[int(len(vol_collection)/2)])

higest opening price is 53.11
lowest opening price is 34.0
largest change in one day is 2.8100000000000023
largest change between 2 days is 2.559999999999995
average trading volume, based on 365 days and not opening days 62264.94794520548
median trading volume is 76286.0
